<a href="https://colab.research.google.com/github/Almonfrey/MAI-Course/blob/main/class6_practical_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [ ]:
# Standard imports for data analysis
import pandas as pd  # Data processing
import numpy as np  # Numerical computing

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Matplotlib configuration for inline display (Jupyter only)
%matplotlib inline

Load data

In [ ]:
# Load the housing dataset and display the first 5 rows
df = pd.read_csv("data/us_house_Sales_data.csv")
print(df.head())

        Price                              Address         City  Zipcode  \
0    $554,217    5926 Oak Ave, San Diego, CA 65383    San Diego    65383   
1    $164,454       9583 Oak Ave, Fresno, IL 79339       Fresno    79339   
2  $1,249,331   8224 Oak Ave, Sacramento, TX 87393   Sacramento    87393   
3    $189,267        232 Oak Ave, Fresno, TX 38666       Fresno    38666   
4    $465,778  5446 Pine Rd, Los Angeles, CA 23989  Los Angeles    23989   

  State Bedrooms Bathrooms Area (Sqft)   Lot Size  Year Built  Days on Market  \
0    CA    1 bds      3 ba    772 sqft  4757 sqft        1959             101   
1    IL    1 bds      1 ba   2348 sqft  3615 sqft        1969              46   
2    TX    6 bds      1 ba   3630 sqft  9369 sqft        1990              59   
3    TX    2 bds      1 ba    605 sqft  8804 sqft        1958             119   
4    CA    3 bds      2 ba   1711 sqft  9260 sqft        2020              26   

  Property Type   MLS ID               Listing Agent    

Data formatting

In [ ]:
import pandas as pd

# Display current data types before formatting
print('Data types BEFORE formatting:')
print(df.dtypes)

df2 = df.copy()

# 1. Price - remove $ and commas, convert to float
df2['Price'] = pd.to_numeric(
    df2['Price'].str.replace(r'[$\dollar$,]', '', regex=True),
    errors='coerce'
)

# 2. Bedrooms - extract number, allow NaN, use nullable Int type
df2['Bedrooms'] = pd.to_numeric(
    df2['Bedrooms'].str.extract(r'(\d+)')[0],
    errors='coerce'
).astype('Int64')

# 3. Bathrooms - same as bedrooms
df2['Bathrooms'] = pd.to_numeric(
    df2['Bathrooms'].str.extract(r'(\d+)')[0],
    errors='coerce'
).astype('Int64')

# 4. Area (Sqft) - remove text and commas, convert to float
df2['Area (Sqft)'] = pd.to_numeric(
    df2['Area (Sqft)'].str.replace(r'[^\d.]', '', regex=True),
    errors='coerce'
)

# 5. Lot Size - same as area
df2['Lot Size'] = pd.to_numeric(
    df2['Lot Size'].str.replace(r'[^\d.]', '', regex=True),
    errors='coerce'
)

# Verify formatting results
print('\nData types AFTER formatting:')
print(df2[['Price', 'Bedrooms', 'Bathrooms', 'Area (Sqft)', 'Lot Size']].dtypes)

Data types BEFORE formatting:
Price             object
Address           object
City              object
Zipcode            int64
State             object
Bedrooms          object
Bathrooms         object
Area (Sqft)       object
Lot Size          object
Year Built         int64
Days on Market     int64
Property Type     object
MLS ID            object
Listing Agent     object
Status            object
Listing URL       object
dtype: object

Data types AFTER formatting:
Price          float64
Bedrooms         int64
Bathrooms        int64
Area (Sqft)    float64
Lot Size       float64
dtype: object


/tmp/ipython-input-93704821.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2['Price'] = df2['Price'].str.replace(r'[$\dollar$,]', '', regex=True).replace('', np.nan).astype(float)
